## About Dataset

Reddit is a discussion website which users can post images and text in a subforum called subreddit which users can discuss about shared contents in comment section. This dataset contains 05/2015 comment submissions from reddit users with 54.504.410 rows and 22 columns.

I got my data from kaggle unfornutely this dataset is too big to run on kaggle so I needed to download it.
> https://www.kaggle.com/reddit/reddit-comments-may-2015/notebooks

If you want a JSON format of this data you can download it from: https://files.pushshift.io/reddit/comments/

## Accessing data from sqlite and cleaning it

Used this sqlite query to clean the dataset before extracting it to csv because it caused problems while trying to import the data

I didn't import authorflaircss_class field because it is not important for our analysis

```sqlite
create table reddit_2015_05 as
select 
rd.created_utc,
rd.ups,
rd.subreddit_id,
rd.link_id,
rd.name,
rd.score_hidden,
replace(
	replace(
		replace(
				replace(
					replace(
						replace(
							replace(rd.author_flair_text,'\','')
						,'*','')
					,'#','')
				, X'0A', ' ')
		,char(13),' ')
	,';','')
,'"','') as author_flair_text,
rd.subreddit,
rd.id,
rd.removal_reason,
rd.gilded,
rd.downs,
rd.archived,
rd.author,
rd.score,
rd.retrieved_on,
replace(
	replace(
		replace(
				replace(
					replace(
						replace(
							replace(rd.body,'\','')
						,'*','')
					,'#','')
				, X'0A', ' ')
		,char(13),' ')
	,';','')
,'"','') as body,
rd.distinguished,
rd.edited,
rd.controversiality,
rd.parent_id
from may2015 rd;
```

## Splitting csv data to make it ready for import

I needed to split my csv file so I can import it to PostgreSQL because PostgreSQL copy command doesn't support files bigger than 4GB

I used [csvsplitter](https://www.erdconcepts.com/dbtoolbox/csvsplitter/csvsplitter.zip) from [erdconcepts](https://www.erdconcepts.com/dbtoolbox.html)

Opened up cmd and inserted these lines;

```cmd
cd C:\data\reddit\csvsplitter

CSVSplitter.exe filename="C:\data\reddit\reddit_2015_05.csv" rowcount=5000000
```

It spliced my csv to 11 files ranging from 1.2GB to 1.5GB

## Creating table in PostgreSQL to import our dataset

I created my PostgreSQL table with this query

```PostgreSQL
CREATE TABLE "ODS"."EXT_REDDIT_COMMENTS"
(
    created_utc integer,
    ups integer,
    subreddit_id text COLLATE pg_catalog."default",
    link_id text COLLATE pg_catalog."default",
    name text COLLATE pg_catalog."default",
    score_hidden text COLLATE pg_catalog."default",
    author_flair_text text COLLATE pg_catalog."default",
    subreddit text COLLATE pg_catalog."default",
    id text COLLATE pg_catalog."default",
    removal_reason text COLLATE pg_catalog."default",
    gilded integer,
    downs integer,
    archived text COLLATE pg_catalog."default",
    author text COLLATE pg_catalog."default",
    score integer,
    retrieved_on integer,
    body text COLLATE pg_catalog."default",
    distinguished text COLLATE pg_catalog."default",
    edited text COLLATE pg_catalog."default",
    controversiality integer,
    parent_id text COLLATE pg_catalog."default"
)

TABLESPACE pg_default;

ALTER TABLE "ODS"."EXT_REDDIT_COMMENTS"
    OWNER to postgres;
```

## Importing dataset

Then used PostgreSQL copy command to import my data;

```PostgreSQL
SET STATEMENT_TIMEOUT TO 3000000;

COPY "ODS"."EXT_REDDIT_COMMENTS" FROM 'C:/data/reddit/REDDIT_2015_05-000.CSV' DELIMITER ';';

COPY "ODS"."EXT_REDDIT_COMMENTS" FROM 'C:/data/reddit/REDDIT_2015_05-001.CSV' DELIMITER ';';

COPY "ODS"."EXT_REDDIT_COMMENTS" FROM 'C:/data/reddit/REDDIT_2015_05-002.CSV' DELIMITER ';';

COPY "ODS"."EXT_REDDIT_COMMENTS" FROM 'C:/data/reddit/REDDIT_2015_05-003.CSV' DELIMITER ';';

COPY "ODS"."EXT_REDDIT_COMMENTS" FROM 'C:/data/reddit/REDDIT_2015_05-004.CSV' DELIMITER ';';

COPY "ODS"."EXT_REDDIT_COMMENTS" FROM 'C:/data/reddit/REDDIT_2015_05-005.CSV' DELIMITER ';';

COPY "ODS"."EXT_REDDIT_COMMENTS" FROM 'C:/data/reddit/REDDIT_2015_05-006.CSV' DELIMITER ';';

COPY "ODS"."EXT_REDDIT_COMMENTS" FROM 'C:/data/reddit/REDDIT_2015_05-007.CSV' DELIMITER ';';

COPY "ODS"."EXT_REDDIT_COMMENTS" FROM 'C:/data/reddit/REDDIT_2015_05-008.CSV' DELIMITER ';';

COPY "ODS"."EXT_REDDIT_COMMENTS" FROM 'C:/data/reddit/REDDIT_2015_05-009.CSV' DELIMITER ';';

COPY "ODS"."EXT_REDDIT_COMMENTS" FROM 'C:/data/reddit/REDDIT_2015_05-010.CSV' DELIMITER ';';

COMMIT;
```

## Analyzing our data

Original dataset is too big to handle(54.504.410 rows with 33.3GB size) maybe we should check if it is possible to reduce our data while not affecting our analysis

```PostgreSQL
SELECT 
COUNT(*)                       
FROM "ODS"."EXT_REDDIT_COMMENTS" ERS
WHERE 1=1
AND LENGTH(ERS.BODY) > 2;
```
This query reduces our data to 54.333.604 rows while removing comments like 'OK'

```PostgreSQL
SELECT 
COUNT(*)                       
FROM "ODS"."EXT_REDDIT_COMMENTS" ERS
WHERE 1=1
AND LENGTH(ERS.BODY) > 2
AND (LOWER(ERS.AUTHOR) LIKE '%\_bot\_%'
OR LOWER(ERS.AUTHOR) LIKE '%\-bot\-%');
```
This would remove 958 bot comments with comment author names contains "-bot-" or "_bot_" so it is not that huge decrease

```PostgreSQL
SELECT
COUNT(*)
FROM "ODS"."EXT_REDDIT_COMMENTS" ERS
WHERE 1=1
AND LENGTH(ERS.BODY) > 2
AND NOT (LOWER(ERS.AUTHOR) LIKE '%\_bot\_%'
OR LOWER(ERS.AUTHOR) LIKE '%\-bot\-%')
AND NOT(LOWER(REPLACE(ERS.BODY,'''',''))) LIKE '%im a bot%';
```
We could also filter comments with "I'm a bot" text, this also decreases dataset with 24.918 rows

```PostgreSQL
SELECT
COUNT(*)
FROM "ODS"."EXT_REDDIT_COMMENTS" ERS
WHERE 1=1
AND LENGTH(ERS.BODY) > 2
AND NOT (LOWER(ERS.AUTHOR) LIKE '%\_bot\_%'
OR LOWER(ERS.AUTHOR) LIKE '%\-bot\-%')
AND NOT(LOWER(REPLACE(ERS.BODY,'''','')) LIKE '%im a bot%')
AND ERS.BODY <> '[deleted]'; --3138587
```
This query removes deleted comments which is 3.138.587 rows

```PostgreSQL
SELECT
COUNT(*)
FROM "ODS"."EXT_REDDIT_COMMENTS" ERS
WHERE 1=1
AND LENGTH(ERS.BODY) > 2
AND NOT (LOWER(ERS.AUTHOR) LIKE '%\_bot\_%'
OR LOWER(ERS.AUTHOR) LIKE '%\-bot\-%')
AND NOT(LOWER(REPLACE(ERS.BODY,'''','')) LIKE '%im a bot%')
AND ERS.BODY <> '[deleted]'
AND LENGTH(ERS.REMOVAL_REASON) = 0;
```
We should also remove removed comments which is replaced by removal reason instead of original comments.

```PostgreSQL
SELECT
COUNT(*)
FROM "ODS"."EXT_REDDIT_COMMENTS" ERS
WHERE 1=1
AND LENGTH(ERS.BODY) > 2
AND NOT (LOWER(ERS.AUTHOR) LIKE '%\_bot\_%'
OR LOWER(ERS.AUTHOR) LIKE '%\-bot\-%')
AND NOT(LOWER(REPLACE(ERS.BODY,'''','')) LIKE '%im a bot%')
AND ERS.BODY <> '[deleted]'
AND LENGTH(ERS.removal_reason) = 0
AND ERS.BODY LIKE '% %';
```
We should remove single word comments(1.885.966 rows) because they are not important for our analysis.

```PostgreSQL
SELECT
COUNT(*)
FROM "ODS"."EXT_REDDIT_COMMENTS" ERS
WHERE 1=1
AND LENGTH(ERS.BODY) > 2
AND NOT (LOWER(ERS.AUTHOR) LIKE '%\_bot\_%'
OR LOWER(ERS.AUTHOR) LIKE '%\-bot\-%')
AND NOT(LOWER(REPLACE(ERS.BODY,'''','')) LIKE '%im a bot%')
AND ERS.BODY <> '[deleted]'
AND LENGTH(ERS.removal_reason) = 0
AND ERS.BODY LIKE '% %'
AND ERS.AUTHOR <> 'AutoModerator';
```
286.444

```PostgreSQL
SELECT
COUNT(*)
FROM "ODS"."EXT_REDDIT_COMMENTS" ERS
WHERE 1=1
AND LENGTH(ERS.BODY) > 2
AND NOT(LOWER(ERS.AUTHOR) LIKE '%\_bot\_%' OR LOWER(ERS.AUTHOR) LIKE '%\-bot\-%')
AND NOT(LOWER(REPLACE(ERS.BODY,'''','')) LIKE '%im a bot%')
AND ERS.BODY <> '[deleted]'
AND LENGTH(ERS.removal_reason) = 0
AND ERS.BODY LIKE '% %'
AND ERS.AUTHOR <> 'AutoModerator'
AND ERS.AUTHOR <> '[deleted]'
```
305.983

In [1]:
import pandas as pd
import numpy as np
import psycopg2

conn_string = 'host={pghost} port={pgport} dbname={pgdatabase} user={pguser} password={pgpassword}'.format(pgdatabase='MEF-BDA-PROD',pguser='postgres',pgpassword='123',pghost='localhost',pgport='5432')
conn=psycopg2.connect(conn_string)
cur=conn.cursor()

def check_if_table_exists(schema,table):
    cur.execute("select exists(select * from information_schema.tables where table_schema='{schema}' AND table_name='{table}')".format(schema=schema, table=table))
    return cur.fetchone()[0]

def(check_if_index_exist)(index):
    cur.execute("SELECT EXISTS(SELECT * FROM PG_CLASS WHERE relname = {index})".format(index=index))
    return cur.fetchone()[0]

if(check_if_table_exists('EDW','DWH_REDDIT_COMMENTS')):
    print('Table already exists')   
else:
    cur.execute('set time zone UTC;')
    cur.execute("""
    CREATE TABLE "EDW"."DWH_REDDIT_COMMENTS" AS 
    SELECT
    ROW_NUMBER() OVER (ORDER BY ERS.ID) AS ID,
    TO_TIMESTAMP(GREATEST(ERS.CREATED_UTC ,CAST(ERS.EDITED AS INTEGER))) AS DATE,
    ERS.SUBREDDIT,
    ERS.AUTHOR,
    ERS.AUTHOR_FLAIR_TEXT,
    ERS.SCORE,
    ERS.BODY AS COMMENT
    FROM "ODS"."EXT_REDDIT_COMMENTS" ERS
    WHERE 1=1
    AND LENGTH(ERS.BODY) > 2
    AND NOT(LOWER(ERS.AUTHOR) LIKE '%\_bot\_%' OR LOWER(ERS.AUTHOR) LIKE '%\-bot\-%')
    AND NOT(LOWER(REPLACE(ERS.BODY,'''','')) LIKE '%im a bot%')
    AND ERS.BODY <> '[deleted]'
    AND LENGTH(ERS.removal_reason) = 0
    AND ERS.BODY LIKE '% %'
    AND ERS.AUTHOR <> 'AutoModerator'
    AND ERS.AUTHOR <> '[deleted]';
    """)
    cur.execute('COMMIT;')
    print('Table created')

#sql_command = 'SELECT * FROM "{schema}"."{table}";'.format(schema='EDW', table='DWH_REDDIT_COMMENTS')

#df = pd.read_sql(sql_command,conn)

cur.close()
conn.close()

#print(df.memory_usage(index=True).sum())

#df

Table created


## Sources:

 1. [About Reddit](https://en.wikipedia.org/wiki/Reddit)

 2. [Data source](https://www.kaggle.com/reddit/reddit-comments-may-2015/notebooks)

 3. [Checking if a table exist with psycopg2 on postgreSQL](https://stackoverflow.com/questions/1874113/checking-if-a-postgresql-table-exists-under-python-and-probably-psycopg2)

 4. [Using current time in UTC as default value in PostgreSQL](https://stackoverflow.com/questions/16609724/using-current-time-in-utc-as-default-value-in-postgresql)

 5. [Creating multicolumn index on PostgreSQL](https://www.postgresql.org/docs/9.6/indexes-multicolumn.html)

 6. [Checking if index exist](https://stackoverflow.com/questions/45983169/checking-for-existence-of-index-in-postgresql)